In [4]:
import argparse
from tqdm.auto import tqdm
import torch
import matplotlib.pyplot as plt
from utils import *
import random
import numpy as np
import math
import os 
import scipy
import torch.nn as nn
from modified_stable_diffusion import ModifiedStableDiffusionPipeline
import PIL
from PIL import Image, ImageFilter,ImageEnhance
import commpy.utilities as util
import cv2
from diffusers.models import AutoencoderKL
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='diffusion watermark')
    parser.add_argument('--w_seed', default=0, type=int)
    # parser.add_argument('--dataset', default='Gustavosta/Stable-Diffusion-Prompts')
    parser.add_argument('--dataset', default='coco')
    # parser.add_argument('--dataset', default='stablediffusionDB')
    # parser.add_argument('--model_path', default='../stable-diffusion-2-1-base')
    parser.add_argument('--model_path', default='../stable-diffusion-v1-4')
    parser.add_argument('--image_length', default=512, type=int)
    parser.add_argument('--secret_length', default=48, type=int)
    parser.add_argument('--num_inference_steps', default=25, type=int)
    parser.add_argument('--guidancescale', default=5, type=float)
    parser.add_argument('--reverse_inference_steps', default=25, type=int)
    # parser.add_argument('--model', default='./encoder_decoder_pretrain/model48bit.pth', type=str)
    # parser.add_argument('--model', default='./model48bit_finetuned.pth', type=str)
    parser.add_argument('--model', default='./model48bit_finetuned_backup.pth', type=str)
    args =parser.parse_known_args()[0]
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    torch.set_printoptions(sci_mode=False,profile='full')
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    maxlength=150
    
# dataset
dataset, prompt_key = get_dataset(args)
dataset=promptdataset(dataset,prompt_key)
new_vae=torch.load('./disti.pth').to(device).to(torch.float32)
vae = AutoencoderKL.from_pretrained('../stable-diffusion-v1-4/vae', torch_dtype=torch.float16).to(device)
#model
scheduler = DPMSolverMultistepScheduler.from_pretrained(args.model_path, subfolder='scheduler')
pipe = ModifiedStableDiffusionPipeline.from_pretrained(
        args.model_path,
        scheduler=scheduler,
        torch_dtype=torch.float16,
        revision='fp16',
        )
pipe = pipe.to(device)

#diffusetrace
from encoder_decoder_pretrain.watermark_model import *
encoder=Watermark(secret_length=args.secret_length).to(device)
if args.model !=None:
    encoder.load_state_dict(torch.load(args.model))
encoder.eval()

secret= torch.ones(48, dtype=torch.float32)

/tmp/ipykernel_229684/682822630.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(args.model))


In [5]:
def showlatent(latents,channel):
      matrix = np.array(latents[0, channel].detach().cpu())
      fig, axs = plt.subplots(1, 1, figsize=(5, 5))
      im1=axs.imshow(matrix, cmap='viridis', interpolation='nearest')
      axs.axis('off')
      fig.colorbar(im1).remove()
      fig.tight_layout(pad=0, h_pad=0,rect=(0.1, 0.1, 0.9, 0.9))
      canvas = fig.canvas
      canvas.draw()
      w, h = fig.canvas.get_width_height()
      buf = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
      buf.shape = (w, h, 4)
      buf = np.roll(buf, 3, axis=2)
      pil_image = Image.frombytes("RGBA", (w, h), buf.tobytes())
      plt.close()
      return pil_image

def Watermark_Generation(seed):
    generator = torch.Generator().manual_seed(seed)
    random_latents=get_random_latents(pipe,args,generator=generator)
    args.random_latents=random_latents
    bina = torch.Tensor(secret).unsqueeze(-1).unsqueeze(-1).unsqueeze(0).to(device)
    bina = bina.expand(-1,-1,64,64)
    matrix1,mean,logvar=encoder(bina)
    mean=mean.reshape(-1,4,64,64)
    logvar=logvar.reshape(-1,4,64,64)
    eps = torch.randn_like(logvar)
    std = torch.exp(logvar / 2)
    matrix = eps * std + mean
    init_latents=matrix.half()
    args.init_latents=init_latents
    ret=[]
    rad=[]
    for i in range(4):
        rad.append(showlatent(random_latents,i))
        ret.append(showlatent(init_latents,i))
    return ret,rad

def Image_Generation(prompt):
    with torch.no_grad():
                        height,height = 512,512
                        do_classifier_free_guidance = args.guidancescale > 1.0
                        text_embeddings,negative_prompt_embeds = pipe.encode_prompt(
                            prompt, device, 1, do_classifier_free_guidance
                        )
                        if do_classifier_free_guidance:
                            text_embeddings = torch.cat([negative_prompt_embeds, text_embeddings])
                        pipe.scheduler.set_timesteps(args.num_inference_steps, device=device)
                        timesteps = pipe.scheduler.timesteps
                        latents=args.init_latents
                        lat=[]
                        for i, t in enumerate(timesteps):
                                latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
                                latent_model_input = pipe.scheduler.scale_model_input(latent_model_input, t)
                                noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings)[0]
                                if do_classifier_free_guidance:
                                    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                                    noise_pred = noise_pred_uncond + args.guidancescale * (noise_pred_text - noise_pred_uncond)
                                latents = pipe.scheduler.step(noise_pred, t, latents).prev_sample
                                tmp=(latents / 0.18215).to(next(iter(pipe.vae.post_quant_conv.parameters())).dtype)
                                tmp=new_vae.decode(tmp.float(),args.init_latents.float(), return_dict=False)[0].to(device)
                                lat.append(pipe.image_processor.postprocess(tmp.float().detach(), output_type='pil')[0] )
                        latents = latents / 0.18215     
                        latents = latents.to(next(iter(pipe.vae.post_quant_conv.parameters())).dtype)
                        
                        lat_fine = new_vae.decode(latents.float(),args.init_latents.float(), return_dict=False)[0].to(device)
                        img_fine = pipe.image_processor.postprocess(lat_fine.float().detach(), output_type='pil')[0] 
    return img_fine,lat
    
def prompt_gen(input):
    return input

def Watermark_Generation(seed):
    generator = torch.Generator().manual_seed(seed)
    random_latents=get_random_latents(pipe,args,generator=generator)
    args.random_latents=random_latents
    bina = torch.Tensor(secret).unsqueeze(-1).unsqueeze(-1).unsqueeze(0).to(device)
    bina = bina.expand(-1,-1,64,64)
    matrix1,mean,logvar=encoder(bina)
    mean=mean.reshape(-1,4,64,64)
    logvar=logvar.reshape(-1,4,64,64)
    eps = torch.randn_like(logvar)
    std = torch.exp(logvar / 2)
    matrix = eps * std + mean
    init_latents=matrix.half()
    args.init_latents=init_latents
    ret=[]
    rad=[]
    for i in range(4):
        rad.append(showlatent(random_latents,i))
        ret.append(showlatent(init_latents,i))
    return ret,rad

def unauth_Image_Generation(prompt):
    with torch.no_grad():
                        height,height = 512,512
                        do_classifier_free_guidance = args.guidancescale > 1.0
                        text_embeddings,negative_prompt_embeds = pipe.encode_prompt(
                            prompt, device, 1, do_classifier_free_guidance
                        )
                        if do_classifier_free_guidance:
                            text_embeddings = torch.cat([negative_prompt_embeds, text_embeddings])
                        pipe.scheduler.set_timesteps(args.num_inference_steps, device=device)
                        timesteps = pipe.scheduler.timesteps
                        latents=args.random_latents
                        lat=[]
                        for i, t in enumerate(timesteps):
                                latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
                                latent_model_input = pipe.scheduler.scale_model_input(latent_model_input, t)
                                noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings)[0]
                                if do_classifier_free_guidance:
                                    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                                    noise_pred = noise_pred_uncond + args.guidancescale * (noise_pred_text - noise_pred_uncond)
                                latents = pipe.scheduler.step(noise_pred, t, latents).prev_sample
                                tmp=(latents / 0.18215).to(next(iter(pipe.vae.post_quant_conv.parameters())).dtype)
                                tmp=new_vae.decode(tmp.float(),args.random_latents.float(), return_dict=False)[0].to(device)
                                lat.append(pipe.image_processor.postprocess(tmp.float().detach(), output_type='pil')[0] )
                        latents = latents / 0.18215     
                        latents = latents.to(next(iter(pipe.vae.post_quant_conv.parameters())).dtype)
                        
                        lat_fine = new_vae.decode(latents.float(),args.random_latents.float(), return_dict=False)[0].to(device)
                        img_fine = pipe.image_processor.postprocess(lat_fine.float().detach(), output_type='pil')[0] 
    return img_fine,lat

In [6]:
import gradio as gr
block = gr.Blocks().queue()
with block:
        with gr.Row():
           gr.Markdown("分发式场景下的扩散模型版权保护与用户溯源")
        with gr.Column():
                user=gr.Textbox(value=secret.int().tolist(), label="用户ID(分发式场景下为特定用户微调模型组件)")  
        with gr.Row():
            with gr.Column():
              with gr.Accordion("Watermark Generation (Random)", open=True):
                    watermark_seed=gr.Slider(
                        label="Watermark_Seed",
                        minimum=0,
                        maximum=2147483647,
                        step=1,
                        randomize=True,
                    )
        with gr.Row():
           with gr.Column():
               Original_latent = gr.Gallery(label="随机初始隐变量", show_label=True,
                    columns=[2], rows=[2], object_fit="contain")
           with gr.Column():
               Watermark_latent = gr.Gallery(label="带水印的初始隐变量", show_label=True,
                    columns=[2], rows=[2], object_fit="contain")
               
        with gr.Row():       
                watermark_button = gr.Button(value="水印生成")
                
        with gr.Row():
                with gr.Column():
                    prompt_Input = gr.Textbox(label="提示词输入") 
                with gr.Column():
                    prompt = gr.Textbox(label="已经被确认的提示词")
        with gr.Row():
           prompt_generation_button = gr.Button(value="生成提示词")
        
        
        
        with gr.Row():
               Generated_Watermark_image = gr.Image(label="生成的图像",show_label=True,
                   interactive="True")
        with gr.Row():
               Generated_images = gr.Gallery(label="图像生成的过程", show_label=True,
                    object_fit="contain")
        with gr.Row():
                  image_generation_button = gr.Button(value="图像生成")
                  
                  
                  
        with gr.Row():
               unauth_Generated_Watermark_image = gr.Image(label="生成的图像",show_label=True,
                   interactive="True")
        with gr.Row():
               unauth_Generated_images = gr.Gallery(label="图像生成的过程", show_label=True,
                    object_fit="contain")
        with gr.Row():
                  unauth_image_generation_button = gr.Button(value="随机隐变量图像生成（未授权用户、希望逃避安全机制的用户）")
                  
        watermark_button.click(fn=Watermark_Generation, inputs= [watermark_seed],
                               outputs=[Original_latent,Watermark_latent])
        prompt_generation_button.click(fn=prompt_gen,inputs=[prompt_Input],outputs=[prompt])
        image_generation_button.click(fn=Image_Generation, inputs= [prompt],\
                        outputs=[Generated_Watermark_image,Generated_images])
        unauth_image_generation_button.click(fn=unauth_Image_Generation, inputs= [prompt],\
                        outputs=[unauth_Generated_Watermark_image,unauth_Generated_images])
block.launch(share=True)   
        

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.32.2, however version 5.0.1 is available, please upgrade.
--------
Running on public URL: https://0df5253d112cfd8e25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
